In [1]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import numpy as np

In [2]:
def plot_globaly(plot_data,title,tcol,figfile,vmin,vmax):
    fig   = plt.figure(figsize=(15,10))
    fsize = 16
    ax    = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-100,40,0,80])
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='0.5'))
    ax.coastlines()
    ax.gridlines()

    plt.pcolormesh(lon,lat,plot_data,transform=ccrs.PlateCarree(),vmin=vmin,vmax=vmax,cmap=cmap)
    cb = plt.colorbar(extend='both',orientation='horizontal')
    cb.ax.tick_params(labelsize=fsize)
    cb.set_label('temperature anomaly ($^\circ$C)',size=fsize)
    plt.title(title,fontsize=fsize,color=tcol)
    
    imin = -75
    imax = 0
    jmin = 15
    jmax = 30
    plt.plot([imin,imin,imax,imax,imin],[jmin,jmax,jmax,jmin,jmin],color='C1',lw=3)
    imin = -75
    imax = 0
    jmin = 30
    jmax = 45
    plt.plot([imin,imin,imax,imax,imin],[jmin,jmax,jmax,jmin,jmin],color='C2',lw=3)
    imin = -75
    imax = 0
    jmin = 45
    jmax = 60
    plt.plot([imin,imin,imax,imax,imin],[jmin,jmax,jmax,jmin,jmin],color='C4',lw=3)
    plt.show()

    fig.savefig(figfile,dpi=100,bbox_inches='tight', pad_inches=0.1)

In [ ]:
datadir = '/gws/nopw/j04/acsis/jmecking/CMIP6/composites/'
figdir  = 'jet_figures/210802'

EXP          = 'piControl'
var_ts       = 'max_lats'
#var_ts       = 'max_wind'
var_field    = 'tos'
season_ts    = 'DJF'
season_field = 'DJF'

infile = (datadir + EXP + '_' + var_ts + '-' + season_ts + '_' + var_field + '-' + season_field +'_std.nc' )

p_models  = ['CESM2-FV2_r1i1p1f1', 'NESM3_r1i1p1f1'] # Problem with these models - need to investigate!
s0_models = []
s1_models = []
s2_models = ['ACCESS-CM2_r1i1p1f1', 'CAS-ESM2-0_r1i1p1f1', 'CNRM-CM6-1_r1i1p1f2', 'CNRM-ESM2-1_r1i1p1f2','GFDL-CM4_r1i1p1f1','MIROC-ES2L_r1i1p1f2',
            'BCC-ESM1_r1i1p1f1', 'CESM2-WACCM_r1i1p1f1', 'CNRM-CM6-1-HR_r1i1p1f2', 'IPSL-CM6A-LR_r1i2p1f1', 'MIROC6_r1i1p1f1', 'MRI-ESM2-0_r1i1p1f1', 'UKESM1-0-LL_r1i1p1f2',
            'HadGEM3-GC31-LL_r1i1p1f1','HadGEM3-GC31-MM_r1i1p1f1']
plot_lag = 0

In [ ]:
# Read in information:

ncid   = Dataset(infile,'r')
lon    = ncid.variables['lon'][:]
lat    = ncid.variables['lat'][:]
lags   = ncid.variables['lags'][:]
models = ncid.variables['models'][:]
ncid.close()

nm = len(models)
nl = np.size(lags)

ll = np.squeeze(np.where(lags == plot_lag))

In [ ]:
# Loop through each model and plot figures and save MMM:

cmap  = plt.get_cmap('seismic',21)
vmin  = -0.5*1.05
vmax  = 0.5*1.05
vmind = -1*1.05
vmaxd = 1*1.05
if var_ts == 'max_lats':
    lmin  = 'southernly jet (< mean)'
    lmax  = 'northernly jet (> mean)'
elif var_ts == 'max_wind':
    lmin  = 'weak jet (< mean)'
    lmax  = 'strong jet (> mean)'

mean_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
min0_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
max0_MMM = np.zeros((np.size(lat),np.size(lon)),'float')

cc = 0
for mm in range(0,nm):
    print(models[mm])
    model = models[mm].split('_')[0]
    ENS   = models[mm].split('_')[1]
    
    ncid      = Dataset(infile,'r')
    data_mean = ncid.variables['mean'][mm,:,:]
    data_min0 = ncid.variables['min_std0'][mm,ll,:,:]
    data_max0 = ncid.variables['max_std0'][mm,ll,:,:]
    ncid.close()

    if models[mm] in s0_models:  # Models with too few years
        print(models[mm] + ' has no fields')
    elif models[mm] in p_models:
        print('problem ' + models[mm])
    else:
        cc = cc + 1
        mean_MMM = mean_MMM + data_mean
        min0_MMM = min0_MMM + data_min0
        max0_MMM = max0_MMM + data_max0
        

mean_MMM = mean_MMM/cc
min0_MMM = min0_MMM/cc
max0_MMM = max0_MMM/cc    
print(cc) 

# Plot multi-model mean:       
plot_globaly(min0_MMM-mean_MMM,
             ('MMM\n' + lmin + ' minus mean\n' + season_ts +
             ' jet, ' + season_field + ' SST lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
             '-' + season_field + '_min-mean_0std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max0_MMM-mean_MMM,
             ('MMM\n' + lmax + ' minus mean\n' + season_ts +
             ' jet, ' + season_field + ' SST lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
             '-' + season_field + '_max-mean_0std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max0_MMM-min0_MMM,
             ('MMM\n' + lmax + ' minus ' + lmin + ' \n' + season_ts +
             ' jet, ' + season_field + ' SST lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
              '-' + season_field + '_max-min_0std_lag' + str(int(lags [ll])) + '_boxes.png'),vmind,vmaxd)

In [ ]:
# Loop through each model and plot figures and save MMM:

cmap  = plt.get_cmap('seismic',21)
vmin  = -0.5*1.05
vmax  = 0.5*1.05
vmind = -1*1.05
vmaxd = 1*1.05
if var_ts == 'max_lats':
    lmin  = 'southernly jet (< mean - std)'
    lmax  = 'northernly jet (> mean + std)'
elif var_ts == 'max_wind':
    lmin  = 'weak jet (< mean - std)'
    lmax  = 'strong jet (> mean + std)'

mean_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
min1_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
max1_MMM = np.zeros((np.size(lat),np.size(lon)),'float')

cc = 0
for mm in range(0,nm):
    print(models[mm])
    model = models[mm].split('_')[0]
    ENS   = models[mm].split('_')[1]
    
    ncid      = Dataset(infile,'r')
    data_mean = ncid.variables['mean'][mm,:,:]
    data_min1 = ncid.variables['min_std1'][mm,ll,:,:]
    data_max1 = ncid.variables['max_std1'][mm,ll,:,:]
    ncid.close()

    if models[mm] in s1_models:  # Models with too few years
        print(models[mm] + ' has no fields')
        tcol = 'C0'
    elif models[mm] in p_models:
        print('problem ' + models[mm])
        tcol = 'C3'
    else:
        tcol = 'k'
        cc = cc + 1
        mean_MMM = mean_MMM + data_mean
        min1_MMM = min1_MMM + data_min1
        max1_MMM = max1_MMM + data_max1
        
mean_MMM = mean_MMM/cc
min1_MMM = min1_MMM/cc
max1_MMM = max1_MMM/cc  
print(cc)   

# Plot multi-model mean: 
plot_globaly(min1_MMM-mean_MMM,
             ('MMM\n' + lmin + ' minus mean\n' + season_ts +
             ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
             '-' + season_field + '_min-mean_1std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max1_MMM-mean_MMM,
             ('MMM\n' + lmax + ' minus mean\n' + season_ts +
             ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
             '-' + season_field + '_max-mean_1std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max1_MMM-min1_MMM,
             ('MMM\n' + lmax + ' minus ' + lmin + ' \n' + season_ts +
             ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',
             (figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
              '-' + season_field + '_max-min_1std_lag' + str(int(lags [ll])) + '_boxes.png'),vmind,vmaxd)

In [ ]:
# Loop through each model and plot figures and save MMM:

cmap  = plt.get_cmap('seismic',21)
vmin  = -0.5*1.05
vmax  = 0.5*1.05
vmind = -1*1.05
vmaxd = 1*1.05
if var_ts == 'max_lats':
    lmin  = 'southernly jet (< mean - 2 std)'
    lmax  = 'northernly jet (> mean + 2 std)'
elif var_ts == 'max_wind':
    lmin  = 'weak jet (< mean - 2 std)'
    lmax  = 'strong jet (> mean + 2 std)'

mean_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
min2_MMM = np.zeros((np.size(lat),np.size(lon)),'float')
max2_MMM = np.zeros((np.size(lat),np.size(lon)),'float')

cc = 0
for mm in range(0,nm):
    print(models[mm])
    model = models[mm].split('_')[0]
    ENS   = models[mm].split('_')[1]
    
    ncid      = Dataset(infile,'r')
    data_mean = ncid.variables['mean'][mm,:,:]
    data_min2 = ncid.variables['min_std2'][mm,ll,:,:]
    data_max2 = ncid.variables['max_std2'][mm,ll,:,:]
    ncid.close()

    if models[mm] in s2_models:  # Models with too few years
        print(models[mm] + ' has no fields')
        tcol = 'C0'
    elif models[mm] in p_models:
        print('problem ' + models[mm])
        tcol = 'C3'
    else:
        tcol = 'k'
        cc = cc + 1
        mean_MMM = mean_MMM + data_mean
        min2_MMM = min2_MMM + data_min2
        max2_MMM = max2_MMM + data_max2
        
mean_MMM = mean_MMM/cc
min2_MMM = min2_MMM/cc
max2_MMM = max2_MMM/cc   
print(cc)

# Plot multi-model mean: 
plot_globaly(min2_MMM-mean_MMM,('MMM\n' + lmin + ' minus mean\n' + season_ts +
    ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',(figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
    '-' + season_field + '_min-mean_2std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max2_MMM-mean_MMM,('MMM\n' + lmax + ' minus mean\n' + season_ts +
             ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',(figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
             '-' + season_field + '_max-mean_2std_lag' + str(int(lags [ll])) + '_boxes.png'),vmin,vmax)

plot_globaly(max2_MMM-min2_MMM,('MMM\n' + lmax + ' minus ' + lmin + '\n' + season_ts +
             ' jet, SST ' + season_field + ' lags ' + str(int(lags[ll])) + ' years'),'k',(figdir + '/' + 'MMM_' + var_ts + '-' + season_ts + '_' + var_field +
              '-' + season_field + '_max-min_2std_lag' + str(int(lags [ll])) + '_boxes.png'),vmind,vmaxd)